In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader


In [2]:
txf = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor()
])

train_data = datasets.Flowers102(
    root='flowers102/train/',
    split='train',
    transform=txf,
    download=True
)

val_data = datasets.Flowers102(
    root='flowers102/val',
    split='val',
    transform=txf,
    download=True
)

test_data = datasets.Flowers102(
    root='flowers102/test',
    split='val',
    transform=txf,
    download=True
)


100%|██████████| 344862509/344862509 [09:15<00:00, 620313.84it/s] 


Extracting flowers102\train\flowers-102\102flowers.tgz to flowers102\train\flowers-102


100%|██████████| 502/502 [00:00<00:00, 250480.68it/s]


100%|██████████| 14989/14989 [00:00<00:00, 7357334.42it/s]


100%|██████████| 344862509/344862509 [08:32<00:00, 673134.05it/s] 


...

100%|██████████| 14989/14989 [00:00<00:00, 7533663.59it/s]


100%|██████████| 344862509/344862509 [07:19<00:00, 784748.29it/s] 


Extracting flowers102\test\flowers-102\102flowers.tgz to flowers102\test\flowers-102


100%|██████████| 502/502 [00:00<00:00, 501797.09it/s]


100%|██████████| 14989/14989 [00:00<00:00, 15596234.84it/s]


In [16]:
torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
val_loader = DataLoader(val_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)

print(f'Training images available: {len(train_data)}')
print(f'Validation images available:  {len(val_data)}')
print(f'Testing images available:  {len(test_data)}')


Training images available: 1020
Validation images available:  1020
Testing images available:  6149


In [48]:
class AlexNet(nn.Module):
    def __init__(self, out_sz, p=0.5) -> None:
        super().__init__()
        self.drop = p

        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=96,
                kernel_size=11,
                stride=4,
            ),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(
                in_channels=96,
                out_channels=256,
                kernel_size=5,
                padding=2
            ),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(
                in_channels=256,
                out_channels=384,
                kernel_size=3,
                padding=1,
            ),
            nn.ReLU(inplace=True),

            nn.Conv2d(
                in_channels=384,
                out_channels=384,
                kernel_size=3,
                padding=1,
            ),
            nn.ReLU(inplace=True),

            nn.Conv2d(
                in_channels=384,
                out_channels=256,
                kernel_size=3,
                padding=1
            ),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Dropout(self.drop)
        )

        self.fc1 = nn.Linear(256*6*6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 300)
        self.fc4 = nn.Linear(300, out_sz)

    def forward(self, x):
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = F.dropout(F.relu(self.fc1(x)), self.drop)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)

        return x

    def interm_features(self, x):
        with torch.no_grad():
            x = self.conv(x)
            x = torch.flatten(x, 1)
            x = F.dropout(F.relu(self.fc1(x)), self.drop)
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            return x


In [49]:
torch.manual_seed(9)
alex = AlexNet(102, 0.5)
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(alex.parameters(), lr=0.0001)


In [50]:
from torchsummary import summary
summary(alex, (3, 227, 227))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
          Dropout-14            [-1, 25

In [51]:
if torch.cuda.is_available():
    print("CUDA available. Using GPU acceleration.")
    device = "cuda"
else:
    print("CUDA is NOT available. Using CPU for training.")
    device = "cpu"


CUDA is NOT available. Using CPU for training.


In [ ]:
try:
    alex.load_state_dict(torch.load('./alex_en.pt'))
except:
    n_train = len(train_data)
    n_val = len(val_data)
    e = 500

    train_losses = []
    val_losses = []
    train_accuracy = []
    val_accuracy = []

    for i in range(e):
        trn_corr = 0
        val_corr = 0

        for b, (X_train, y_train) in enumerate(train_loader):
            y_pred = alex(X_train)
            loss = criterion(y_pred, y_train)

            predicted = torch.max(y_pred.data, 1)[1]
            batch_corr = (predicted == y_train).sum()
            trn_corr += batch_corr

            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

        train_losses.append(loss.item())
        train_accuracy.append(trn_corr*100/n_train)

        with torch.no_grad():
            for b, (X_val, y_val) in enumerate(val_loader):
                y_val = alex(X_val)
                predicted = torch.max(y_val.data, 1)[1]
                val_corr += (predicted == y_val).sum()

        loss = criterion(y_val, y_val)
        val_losses.append(loss.item())
        val_accuracy.append(val_corr*100/n_val)

        print(
            f'epoch: {i+1}\ttrain loss: {train_losses[i]:.3f} \ttrain accuracy: {train_accuracy[i]:2.3f}%')
        print(
            f"\t\tval loss:  {val_losses[i]:.3f} \tval accuracy:  {val_accuracy[i]:2.3f}%\n")


In [ ]:
all_interm_features_train = []
train_labels = []
for b, (x, y) in enumerate(train_loader):
    all_interm_features_train.append(alex.interm_features(x))
    train_labels.append(y.view(10, 1))

train_features = torch.concatenate(all_interm_features_train, axis=0)
train_labels = torch.concatenate(train_labels, axis=0)

all_interm_features_test = []
test_labels = []
for b, (x, y) in enumerate(test_loader):
    all_interm_features_test.append(alex.interm_features(x))
    test_labels.append(y.view(10, 1))

test_features = torch.concatenate(all_interm_features_test, axis=0)
test_labels = torch.concatenate(test_labels, axis=0)

train_df = pd.DataFrame(train_features)
train_df['y'] = train_labels
test_df = pd.DataFrame(test_features)
test_df['y'] = test_labels

train_df.to_csv('alex_train.csv', index=False)
test_df.to_csv('alex_test.csv', index=False)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
df_train = pd.read_csv('alex_train.csv')
df_test = pd.read_csv('alex_test.csv')

X_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values
X_test = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = RandomForestClassifier(
    n_estimators=50,
    criterion='entropy',
    random_state=0
)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"Accuracy with Random Forest: {round(acc*100, 2)}%")
